In [1]:
import pandas as pd
from ydata_profiling import ProfileReport
import os

caminho_dados = r"C:\Users\gabri\OneDrive\Belgeler\GitHub\Python\AnalisesRedebras\Dados\RelPedidoCli.xls"

df_comercial = pd.read_excel(caminho_dados)

In [2]:
df_comercial

,PEDIDO,EMP,TIPO,DATA,NOME,INTEGRAÇÃO,UF,FORMA PGTO,OUTRAS DESP,VLR. FRETE,...,VENDEDOR,NF,NF. EMISSAO,DT. RETIRADA,FIN,REPRESENTANTE,CIDADE,STATUS PEDIDO,FRETE TYPE,MKP
0,218756,1.0,PEDIDO,25/11/2025,FLUXOVAL ACESSORIOS HIDRAULICOS INDUSTRIAIS LTDA,218756.0,MG,28,0.0,0.00,...,GERSON,108428.0,26/11/2025,NaN,BX,NaN,CONTAGEM,PENDENTE,NaN,"50,00 %"
1,218791,1.0,PEDIDO,03/11/2025,GP FLUIDOS BOMBAS QUIMICAS LTDA,218791.0,SP,A VISTA,0.0,0.00,...,FELIPE,107859.0,06/11/2025,NaN,BX,NaN,SAO PAULO,PENDENTE,NaN,"140,28 %"
2,219090,1.0,PEDIDO,03/11/2025,CENTRAL VALVULAS INDUSTRIA E COMERCIO DE EQUIP...,219090.0,SP,28,0.0,0.00,...,EUDES,107745.0,04/11/2025,NaN,BX,NaN,PIRACICABA,PENDENTE,NaN,"50,00 %"
3,219415,1.0,PEDIDO,14/11/2025,SCIENTECH AMBIENTAL INDUSTRIA E COMERCIO LTDA,219415.0,SP,30,NaN,NaN,...,EUDES,108171.0,17/11/2025,NaN,BX,NaN,SAO PAULO,PENDENTE,NaN,"78,96 %"
4,219449,1.0,PEDIDO,04/11/2025,FLUXOVAL ACESSORIOS HIDRAULICOS INDUSTRIAIS LTDA,219449.0,MG,28,0.0,0.00,...,GERSON,107822.0,05/11/2025,NaN,BX,NaN,CONTAGEM,PENDENTE,NaN,"50,00 %"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,224836,1.0,PEDIDO,28/11/2025,ANTONIO DA SILVA PAES MANUTENCAO,224836.0,SP,28/35/42,0.0,0.00,...,GERSON,108596.0,28/11/2025,NaN,BX,NaN,ITAQUAQUECETUBA,PENDENTE,NaN,"-30,42 %"
652,224848,1.0,PEDIDO,28/11/2025,NISSIN FOODS DO BRASIL LTDA,224848.0,PR,30,0.0,0.00,...,ANDERSON,108627.0,01/12/2025,NaN,BX,NaN,PONTA GROSSA,PENDENTE,NaN,"45,75 %"
653,224853,1.0,PEDIDO,28/11/2025,GRATT INDUSTRIA DE MAQUINAS LTDA,224853.0,SC,45,0.0,0.00,...,EUDES,108661.0,02/12/2025,NaN,BX,NaN,CAPINZAL,PENDENTE,NaN,"134,55 %"
654,224854,1.0,PEDIDO,28/11/2025,PABLO NUNES DE JESUS SOUZA,111.0,CE,LOJA INTEGRADA,0.0,125.38,...,ECOMMERCE,108620.0,01/12/2025,NaN,BX,NaN,FORTALEZA,PENDENTE,NaN,"107,60 %"


In [6]:
colunas_preço = ['TOTAL CUSTO', 'TOTAL ITENS', 'TOTAL ITENS C/ IPI', 'TOTAL PEDIDO']

for col in colunas_preço:
    # 1. Remove o ponto do milhar (ex: 29.901,72 -> 29901,72)
    df_comercial[col] = df_comercial[col].astype(str).str.replace('.', '', regex=False)
    
    # 2. Troca a vírgula pelo ponto decimal (ex: 29901,72 -> 29901.72)
    df_comercial[col] = df_comercial[col].str.replace(',', '.', regex=False)
    
    # 3. Converte para número (float)
    df_comercial[col] = pd.to_numeric(df_comercial[col], errors='coerce')

In [ ]:
# Modificando as colunas de Data para o tipo Data
colunas_data = ['DATA', 'NF. EMISSAO']

for col in colunas_data:
    df_comercial[col] = pd.to_datetime(df_comercial[col], dayfirst=True, errors='coerce')

ValueError: to assemble mappings requires at least that [year, month, day] be specified: [day,month,year] is missing